In [1]:
from sklearn.metrics import ndcg_score, dcg_score
import json
import math
import numpy as np
wordEmbedDataFile = 'drive/MyDrive/irProject/wordEmbedDataFile.json'
tfidfDataFile = 'drive/MyDrive/irProject/tfidfDataFile.json'
kmeansDataFile = 'drive/MyDrive/irProject/kmeansDataFile.json'
knnDataFile = 'drive/MyDrive/irProject/knnModel.json'
spotifyFile = 'drive/MyDrive/irProject/baselineData.json'

def computeIndividualNDCG(modelData, spotifyData):
  spotifyRelevance = {}
  # go through spotify data and make relevance table
  for key in spotifyData:
    # make an entry with a dict containing the relevance ids with the id as a key
    currDict = {}
    spotifyList = spotifyData[key]
    fourList = []
    threeList = []
    twoList = []
    oneList = []

    sections = math.floor(len(spotifyList)/4)
    for i in range(sections):
      fourList.append(spotifyList[i][0])

    currDict[4] = fourList
    for i in range(sections, sections *2):
      threeList.append(spotifyList[i][0])

    for i in range(sections * 2, sections * 3):
      twoList.append(spotifyList[i][0])
    for i in range(sections * 3, sections * 4):
      oneList.append(spotifyList[i][0])
    
    currDict[3] = threeList
    currDict[2] = twoList
    currDict[1] = oneList

    spotifyRelevance[key] = currDict 

  # for every key in the model data, we are going to compute ndcg with spotify data as ground truth
  total = 0.0
  for key in modelData:
    individualSongList = []
    # do each song and make a list
    goldData = spotifyRelevance[key]
    for result in modelData[key]:
      if (result[0] in goldData[4]):
        individualSongList.append(4)
      elif (result[0] in goldData[3]):
        individualSongList.append(3)
      elif (result[0] in goldData[2]):
        individualSongList.append(2)
      elif (result[0] in goldData[1]):
        individualSongList.append(1)
      else:
        individualSongList.append(0)

    sortedList = sorted(individualSongList)
    sortedList.reverse()
    # we have groud-truth relevance of some answers to a query:
    true_relevance = np.asarray([sortedList])
    # we predict some scores (relevance) for the answers
    scores = np.asarray([individualSongList])

    total = total + ndcg_score(true_relevance, scores)
    #print(true_relevance)

  #print(total)
  #print(len(modelData.keys()))
  finalNdcg = total/(len(modelData.keys()))
  print(finalNdcg)

ourData = {}
spotifyData = {}

with open(knnDataFile) as json_file:
  ourData = json.load(json_file)

with open(spotifyFile) as spotify_file:
  spotifyData = json.load(spotify_file)

computeIndividualNDCG(ourData, spotifyData)

0.07126916372933177


In [2]:
kmeanData = {}
with open(kmeansDataFile) as json_file:
  kmeanData = json.load(json_file)

computeIndividualNDCG(kmeanData, spotifyData)

0.051963490798544704


In [3]:
tfidfData = {}
with open(tfidfDataFile) as json_file:
  tfidfData = json.load(json_file)

computeIndividualNDCG(tfidfData, spotifyData)

0.04396234302915192


In [4]:
wordEmbedData = {}
with open(wordEmbedDataFile) as json_file:
  wordEmbedData = json.load(json_file)

computeIndividualNDCG(wordEmbedData, spotifyData)

0.1232077961326378
